In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers

import tensorflow as tf
from typing import NamedTuple, Optional
from sklearn import preprocessing 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import resample
from sklearn.utils import shuffle
     
import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

2024-08-09 17:42:41.990847: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-09 17:42:41.990971: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-09 17:42:42.155952: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
df = pd.read_csv('/kaggle/input/isic-2024-challenge/train-metadata.csv')
df_test = pd.read_csv('/kaggle/input/isic-2024-challenge/test-metadata.csv')

/tmp/ipykernel_24/2653809078.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/kaggle/input/isic-2024-challenge/train-metadata.csv')


In [3]:
df_2018 = pd.read_csv('/kaggle/input/isic-2018-jpg-224x224-resized/train-metadata.csv')
df_2019 = pd.read_csv('/kaggle/input/isic-2019-jpg-224x224-resized/train-metadata.csv')
df_2020 = pd.read_csv('/kaggle/input/isic-2020-jpg-224x224-resized/train-metadata.csv')

In [4]:
# df_2019.head()

## Constants

In [5]:
RANDOM_STATE = 42
IMAGE_SIZE=224
BATCH_SIZE = 32
# METRICS = [
#       keras.metrics.TruePositives(name='tp'),
#       keras.metrics.FalsePositives(name='fp'),
#       keras.metrics.TrueNegatives(name='tn'),
#       keras.metrics.FalseNegatives(name='fn'),
#       keras.metrics.AUC(name='auc'),
#       keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
# ]
# img_augmentation_layers = [
#     layers.RandomRotation(factor=0.15),
#     layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
#     layers.RandomFlip(),
#     layers.RandomContrast(factor=0.1),
# ]




## Helper functions

In [6]:

def add_path(path,dir):
    return f"{dir}/{path}.jpg"



def get_sample(df,ratio):
    df_positive = df[df['target']==1].reset_index(drop=True)
    df_negative = df[df['target']==0].reset_index(drop=True)
    df_negative= resample(df_negative,n_samples=len(df_positive)*ratio,random_state=RANDOM_STATE)
#     df_positive = resample(df_positive,n_samples=len(df_positive),random_state=RANDOM_STATE)
    
    df_concat = pd.concat([df_positive,df_negative])
    df_concat = shuffle(df_concat).reset_index(drop=True)
    
    print(f'positive shape: {df_positive.shape}, negative shape: {df_negative.shape}')
    print(f'Full dataset shape: {df_concat.shape}')
    
    return df_concat

def visualize(df,idx):
    images = []
    labels = []
    for id in idx:
        current_row = df.iloc[id]
        images.append(plt.imread(current_row["file_path"]))
        labels.append(current_row["target"])
    
    for i, (image, label) in enumerate(zip(images, labels), start=1):
        plt.subplot(3, 3, i)  # Adjusted to start from 1
        plt.imshow(image)
        plt.axis("off")
        plt.title(label)
    plt.show()

In [7]:
# def encode(df, columns):
#     label_encoder = preprocessing.LabelEncoder() 
#     for col in columns:
#         df[col]= label_encoder.fit_transform(df[col])
#     return df

# def scale_columns(df, columns):
#     scaler = preprocessing.StandardScaler()
    
#     # Transform each specified column
#     for col in columns:
#         df[col] = scaler.fit_transform(df[[col]])
        
#     return df

# class Metadata(NamedTuple):
#     image: Optional[tf.Tensor] = None
#     metadata: Optional[tf.Tensor] = None


# def img_augmentation(images):
#     for layer in img_augmentation_layers:
#         images = layer(images)
#     return images

# @tf.function(input_signature=[
#     tf.TensorSpec(shape=[], dtype=tf.string, name='image_path'),
# ])
# def preprocess_image(path):
#     image = tf.io.read_file(path)
#     image = tf.image.decode_jpeg(image, channels=3)
#     image = tf.image.resize(image, IMAGE_SIZE)
#     image = image / 255.0
# #     image = img_augmentation(image)
#     return image

# def preprocess_metadata(filtered_dict):
#     # Convert metadata to tensor
#     tensor_dict = tf.stack(list(filtered_dict.values()), axis=-1)
#     return tensor_dict

# def load_image_and_metadata(metadata, image_size):
#     image = preprocess_image(metadata['file_path'], image_size)
#     filtered_dict = {k: v for k, v in metadata.items() if k != 'file_path'}
#     metadata_tensor = preprocess_metadata(filtered_dict)
#     return Metadata(image=image, metadata=metadata_tensor)


# def unbatch_test_dataset(test_dataset,model):
#     unbatched_dataset = test_dataset.unbatch()

#     # Initialize lists to store true labels and predictions
#     images = []
#     labels = []


#     # Iterate over the unbatched dataset to get predictions and true labels
#     for img_batch, label_batch in test_dataset:
#         images.append(img_batch)
#         labels.append(label_batch)

#     # Convert lists to numpy arrays
#     images = np.concatenate(images, axis=0)
#     labels = np.concatenate(labels, axis=0)

#     # Make predictions on the entire dataset
#     y_pred = model.predict(images).ravel()  # Flatten predictions if necessary

#     # Convert labels to a flat array
#     y_true = labels.ravel()
#     # Convert lists to numpy arrays
#     y_true = np.array(y_true)
#     y_pred = np.array(y_pred)
#     return y_true, y_pred


In [8]:
# def calculate_weight(pos,neg):

#     # Total number of samples
#     total = pos + neg

#     # Calculate class weights
#     weight_for_0 = (1 / neg) * (total / 2.0)
#     weight_for_1 = (1 / pos) * (total / 2.0)

#     class_weight = {0: weight_for_0, 1: weight_for_1}

#     print('Weight for class 0 (Negative): {:.2f}'.format(weight_for_0))
#     print('Weight for class 1 (Positive): {:.2f}'.format(weight_for_1))
#     return class_weight

In [9]:
# random_choices = random.choices(list(range(df.shape[0])), k=9)
# visualize(df,random_choices)

In [10]:
# def create_dataset(df, image_size=(139, 139), batch_size=32, training=False, return_type='both'):
#     labels = df[['target']]

#     dataset = tf.data.Dataset.from_tensor_slices((df['file_path'].values, labels))
#     dataset = dataset.map(lambda x, y: (preprocess_image(x, image_size), y),
#                               num_parallel_calls=tf.data.experimental.AUTOTUNE)
#     # Shuffle and batch the dataset
#     if training:
#         dataset = dataset.shuffle(len(df))
#     dataset = dataset.batch(batch_size)
#     dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    
#     return dataset


In [11]:
# from sklearn.metrics import roc_auc_score
# import numpy as np

# def compute_partial_auc(y_true, y_pred, min_tpr):
#     """
#     Computes the partial AUC score given true labels, predicted scores, and minimum TPR.

#     Parameters:
#     y_true (np.array): True binary labels.
#     y_pred (np.array): Predicted probabilities or scores.
#     min_tpr (float): Minimum true positive rate (TPR) to consider for partial AUC.

#     Returns:
#     partial_auc (float): The computed partial AUC score.
#     """
#     # Compute the transformed ground truth and predictions
#     v_gt = np.abs(y_true - 1)
#     v_pred = 1.0 - y_pred

#     # Compute the maximum false positive rate (FPR) based on the minimum TPR
#     max_fpr = 1 - min_tpr

#     # Compute the partial AUC using roc_auc_score
#     partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)

#     # Rescale the partial AUC
#     partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) * (partial_auc_scaled - 0.5)

#     return partial_auc

# # Example usage
# # y_true = np.array([...])  # Replace with your true labels
# # y_pred = np.array([...])  # Replace with your predicted scores
# # min_tpr = 0.9  # Example minimum TPR value
# # partial_auc = compute_partial_auc(y_true, y_pred, min_tpr)
# # print(f"Partial AUC: {partial_auc:.4f}")

In [12]:
# def create_dataset(df, image_size=(139, 139), batch_size=32, training=False, return_type='both'):
#     # Extract labels
#     labels = df[['target']]
    
#     # Depending on the return type, prepare the dataset accordingly
#     if return_type == 'both':
#         dataset = tf.data.Dataset.from_tensor_slices((dict(df.drop(columns=['target'])), labels))
#         dataset = dataset.map(lambda x, y: (load_image_and_metadata(x, image_size), y),
#                               num_parallel_calls=tf.data.experimental.AUTOTUNE)
#     elif return_type == 'metadata':
#         dataset = tf.data.Dataset.from_tensor_slices((dict(df.drop(columns=['target'])), labels))
#         dataset = dataset.map(lambda x, y: (preprocess_metadata({k: v for k, v in x.items() if k != 'file_path'}), y),
#                               num_parallel_calls=tf.data.experimental.AUTOTUNE)
#     elif return_type == 'images':
#         dataset = tf.data.Dataset.from_tensor_slices((df['file_path'].values, labels))
#         dataset = dataset.map(lambda x, y: (preprocess_image(x, image_size), y),
#                               num_parallel_calls=tf.data.experimental.AUTOTUNE)
#     else:
#         raise ValueError("Invalid return_type. Must be one of ['both', 'metadata', 'images']")
    
#     # Shuffle and batch the dataset
#     if training:
#         dataset = dataset.shuffle(len(df))
#     dataset = dataset.batch(batch_size)
#     dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    
#     return dataset

# from sklearn.metrics import roc_auc_score
# import numpy as np


class PartialAUC(tf.keras.metrics.Metric):
    def __init__(self, name='partial_auc', **kwargs):
        super(PartialAUC, self).__init__(name=name, **kwargs)
        self.thresholds = tf.linspace(0.0, 1.0, 101)
        self.tp = self.add_weight(name='tp', shape=(101,), initializer='zeros', dtype=tf.float32)
        self.fp = self.add_weight(name='fp', shape=(101,), initializer='zeros', dtype=tf.float32)
        self.fn = self.add_weight(name='fn', shape=(101,), initializer='zeros', dtype=tf.float32)
        self.tn = self.add_weight(name='tn', shape=(101,), initializer='zeros', dtype=tf.float32)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.reshape(tf.cast(y_true, tf.float32), (-1, 1))
        y_pred = tf.cast(y_pred, tf.float32)

        # Debugging shapes
        #tf.print("reshaped y_true shape:", tf.shape(y_true))
        #tf.print("y_pred shape:", tf.shape(y_pred))

        thresholds = self.thresholds[tf.newaxis, :]
        preds = tf.cast(y_pred[:, tf.newaxis] >= thresholds, tf.float32)

        tp = tf.reduce_sum(preds * y_true[:, tf.newaxis], axis=0)
        fp = tf.reduce_sum(preds * (1 - y_true)[:, tf.newaxis], axis=0)
        fn = tf.reduce_sum((1 - preds) * y_true[:, tf.newaxis], axis=0)
        tn = tf.reduce_sum((1 - preds) * (1 - y_true)[:, tf.newaxis], axis=0)

        # Check tensor shapes
        #tf.print("tp shape:", tf.shape(tp))
        #tf.print("fp shape:", tf.shape(fp))
        #tf.print("fn shape:", tf.shape(fn))
        #tf.print("tn shape:", tf.shape(tn))

        # Reshape assuming desired shape is (num_thresholds,)
        num_thresholds = tf.shape(self.thresholds)[0]
        
        reshaped_tp = tf.reshape(tp, [num_thresholds])
        reshaped_fp = tf.reshape(fp, [num_thresholds])
        reshaped_fn = tf.reshape(fn, [num_thresholds])
        reshaped_tn = tf.reshape(tn, [num_thresholds])
        
        self.tp.assign_add(reshaped_tp)
        self.fp.assign_add(reshaped_fp)
        self.fn.assign_add(reshaped_fn)
        self.tn.assign_add(reshaped_tn)

    def result(self):
        # Avoid division by zero
        epsilon = tf.keras.backend.epsilon()
        tpr = self.tp / (self.tp + self.fn + epsilon)
        fpr = self.fp / (self.fp + self.tn + epsilon)

        # Sort fpr and corresponding tpr
        sorted_indices = tf.argsort(fpr)
        sorted_fpr = tf.gather(fpr, sorted_indices)
        sorted_tpr = tf.gather(tpr, sorted_indices)

        # Apply mask for TPR >= 0.8
        mask = sorted_tpr >= 0.8
        partial_fpr = tf.boolean_mask(sorted_fpr, mask)
        partial_tpr = tf.boolean_mask(sorted_tpr, mask)

        # Calculate partial AUC using the trapezoidal rule
        def compute_partial_auc():
            delta_fpr = partial_fpr[1:] - partial_fpr[:-1]
            tpr_sum = partial_tpr[1:] + partial_tpr[:-1]
            return tf.reduce_sum(delta_fpr * tpr_sum / 2)

        partial_auc = tf.cond(tf.shape(partial_fpr)[0] > 1,
                              compute_partial_auc,
                              lambda: tf.constant(0.0))

        return partial_auc * 0.2

    def reset_states(self):
        self.tp.assign(tf.zeros_like(self.tp))
        self.fp.assign(tf.zeros_like(self.fp))
        self.fn.assign(tf.zeros_like(self.fn))
        self.tn.assign(tf.zeros_like(self.tn))

# def compute_partial_auc(y_true, y_pred, min_tpr):
#     """
#     Computes the partial AUC score given true labels, predicted scores, and minimum TPR.

#     Parameters:
#     y_true (np.array): True binary labels.
#     y_pred (np.array): Predicted probabilities or scores.
#     min_tpr (float): Minimum true positive rate (TPR) to consider for partial AUC.

#     Returns:
#     partial_auc (float): The computed partial AUC score.
#     """
#     # Compute the transformed ground truth and predictions
#     v_gt = np.abs(y_true - 1)
#     v_pred = 1.0 - y_pred

#     # Compute the maximum false positive rate (FPR) based on the minimum TPR
#     max_fpr = 1 - min_tpr

#     # Compute the partial AUC using roc_auc_score
#     partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)

#     # Rescale the partial AUC
#     partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) * (partial_auc_scaled - 0.5)

#     return partial_auc

# Example usage
# y_true = np.array([...])  # Replace with your true labels
# y_pred = np.array([...])  # Replace with your predicted scores
# min_tpr = 0.9  # Example minimum TPR value
# partial_auc = compute_partial_auc(y_true, y_pred, min_tpr)
# print(f"Partial AUC: {partial_auc:.4f}")


In [13]:
class ClearMemory(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()

clear_memory = ClearMemory()


In [14]:
# df_concat = get_sample(df,ratio=20)

In [15]:
# directory = '/kaggle/input/isic-2024-challenge/train-image'
# df_concat['file_path'] = df_concat['isic_id'].apply(lambda x: add_path(x, directory))

In [16]:
df_2018 = get_sample(df_2018,ratio = 10)
# df_2018['file_path'] = df_2018['isic_id'].apply(lambda x: add_path(x, '/kaggle/input/isic-2019-jpg-224x224-resized/train-image/image'))
df_2019 = get_sample(df_2019,ratio = 10)
# df_2019['file_path'] = df_2019['isic_id'].apply(lambda x: add_path(x, '/kaggle/input/isic-2019-jpg-224x224-resized/train-image/image'))
df_2020 = get_sample(df_2020,ratio = 6)
# df_2020['file_path'] = df_2020['isic_id'].apply(lambda x: add_path(x, '/kaggle/input/isic-2020-jpg-224x224-resized/train-image/image'))

# df_concat_previous = pd.concat([df_2019,df_2020],axis=0)

positive shape: (785, 4), negative shape: (7850, 4)
Full dataset shape: (8635, 4)
positive shape: (4522, 4), negative shape: (45220, 4)
Full dataset shape: (49742, 4)
positive shape: (584, 4), negative shape: (3504, 4)
Full dataset shape: (4088, 4)


In [17]:
# type(df_2018)

In [18]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [19]:
train_datagen_2018 = ImageDataGenerator(rescale=1./255)

In [20]:
df_2018['target'] = df_2018['target'].astype(str)
df_2018['isic_id'] = df_2018['isic_id'] + ".jpg" 

In [21]:
 # Add correct extension if missing


In [22]:
train_2018 = train_datagen_2018.flow_from_dataframe(df_2018,
                                              directory="/kaggle/input/isic-2018-jpg-224x224-resized/train-image/image",
                                              x_col="isic_id",
                                              y_col="target",
                                              target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                              batch_size=BATCH_SIZE,
                                              class_mode='binary',
                                              )

Found 7205 validated image filenames belonging to 2 classes.


/opt/conda/lib/python3.10/site-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 1430 invalid image filename(s) in x_col="isic_id". These filename(s) will be ignored.
  warnings.warn(


In [23]:
df_2019['target'] = df_2019['target'].astype(str)
df_2019['isic_id'] = df_2019['isic_id'] + ".jpg" 

In [24]:
# df_2019['isic_id'].head()

In [25]:
train_datagen_2019 = ImageDataGenerator(rescale=1./255,validation_split=0.2)
# train_datagen_2020 = ImageDataGenerator(rescale=1./255)

In [26]:
# Training set
train_2019 = train_datagen_2019.flow_from_dataframe(
    df_2019,
    directory="/kaggle/input/isic-2019-jpg-224x224-resized/train-image/image",
    x_col="isic_id",
    y_col="target",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'  # Specify that this is the training set
)

# Validation set
val_2019 = train_datagen_2019.flow_from_dataframe(
    df_2019,
    directory="/kaggle/input/isic-2019-jpg-224x224-resized/train-image/image",
    x_col="isic_id",
    y_col="target",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'  # Specify that this is the validation set
)

Found 39794 validated image filenames belonging to 2 classes.
Found 9948 validated image filenames belonging to 2 classes.


In [27]:
df_2020['target'] = df_2020['target'].astype(str)
df_2020['isic_id'] = df_2020['isic_id'] + ".jpg" 
train_datagen_2020 = ImageDataGenerator(rescale=1./255,validation_split=0.2)
# Training set

In [28]:

train_2020 = train_datagen_2020.flow_from_dataframe(
    df_2020,
    directory="/kaggle/input/isic-2020-jpg-224x224-resized/train-image/image",
    x_col="isic_id",
    y_col="target",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'  # Specify that this is the training set
)

# Validation set
val_2020 = train_datagen_2020.flow_from_dataframe(
    df_2020,
    directory="/kaggle/input/isic-2020-jpg-224x224-resized/train-image/image",
    x_col="isic_id",
    y_col="target",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'  # Specify that this is the validation set
)

Found 3271 validated image filenames belonging to 2 classes.
Found 817 validated image filenames belonging to 2 classes.


In [29]:
df = get_sample(df,ratio=20)

positive shape: (393, 55), negative shape: (7860, 55)
Full dataset shape: (8253, 55)


In [30]:
# df.head()

In [31]:
df['target'] = df['target'].astype(str)
df['isic_id'] = df['isic_id'] + ".jpg" 
train_datagen = ImageDataGenerator(rescale=1./255)

In [32]:

# Training set
train_data = train_datagen.flow_from_dataframe(
    df,
    directory="/kaggle/input/isic-2024-challenge/train-image/image",
    x_col="isic_id",
    y_col="target",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training' 
)

# Validation set
# val_data = train_datagen.flow_from_dataframe(
#     df,
#     directory="/kaggle/input/isic-2024-challenge/train-image/image",
#     x_col="isic_id",
#     y_col="target",
#     target_size=(IMAGE_SIZE, IMAGE_SIZE),
#     batch_size=BATCH_SIZE,
#     class_mode='binary',
#     subset='validation'  # Specify that this is the validation set
# )

Found 8253 validated image filenames belonging to 2 classes.


In [33]:
# train_datagen = ImageDataGenerator(rescale=1./255)
# train_set = train_datagen.flow_from_dataframe(df_2018,
#                                               directory="/kaggle/input/isic-2018-jpg-224x224-resized/train-image/image",
#                                               x_col="isic_id",
#                                               y_col="target",
#                                               target_size=(224, 224),
#                                               batch_size=32,
#                                               class_mode='binary',
#                                               shuffle=True)
# print("Number of batches:", len(train_set))  # This should now be > 0


In [34]:
# for data_batch, labels_batch in train_2018:
#     print("data batch shape:", data_batch.shape)
#     print("labels batch shape:", labels_batch.shape)
#     break

In [35]:
# train_set

In [36]:
# df_concat_previous.columns

In [37]:
# df_concat_previous = df_concat_previous[['file_path','target']]

In [38]:
# visualize(df_2020,[1,2,4,5,44,342])

In [39]:
# df_concat_previous.shape

In [40]:
# df_concat_previous.tail()

In [41]:
# directory = '/kaggle/input/isic-2024-challenge/train-image'
# df_concat['file_path'] = df_concat['isic_id'].apply(lambda x: add_path(x, directory))

In [42]:
# df_concat_previous = df_concat_previous[df_concat_previous['file_path'].apply(os.path.exists)]

In [43]:
# df_concat_previous.shape

## Vision Model

In [44]:
# positive = df_concat[df_concat['target']==1]

In [45]:

# augmented_positive = tf.data.Dataset.from_tensor_slices((positive['file_path'].values, positive['target'].values))
# augmented_positive = augmented_positive.map(lambda x, y: (preprocess_image(x), y),
#                               num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [46]:

# augmented_positive1 = tf.data.Dataset.from_tensor_slices((positive['file_path'].values, positive['target'].values))
# augmented_positive1 = augmented_positive1.map(lambda x, y: (preprocess_image(x), y),
#                               num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [47]:
# df_concat['image'] = df_concat['file_path'].apply(preprocess_image)

# dataset = tf.data.Dataset.from_tensor_slices((df_concat['file_path'].values, df_concat['target']))
# dataset = dataset.map(lambda x, y: (preprocess_image(x), y),
#                               num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [48]:
# df_concat_previous.head()

In [49]:
# len(df_concat_previous)

In [50]:
# previous_dataset = tf.data.Dataset.from_tensor_slices((df_concat_previous['file_path'].values, df_concat_previous['target']))
# previous_dataset = previous_dataset.map(lambda x, y: (preprocess_image(x), y))
# previous_dataset = previous_dataset.shuffle(len(df_concat_previous))
# previous_dataset = previous_dataset.batch(BATCH_SIZE)
# previous_dataset = previous_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [51]:
# for data in previous_dataset.take(1):
#     print(data)

In [52]:
# for image,label in previous_dataset.take(1):
#     print(image.shape,label.shape)

In [53]:
# dataset = dataset.concatenate(augmented_positive,name='first')
# dataset = dataset.concatenate(augmented_positive1,name='version2')

In [54]:
# dataset = dataset.shuffle(len(dataset))
# dataset = dataset.batch(BATCH_SIZE)
# dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [55]:
# efficient.get_layer(name='block4c_add').output

In [56]:
# layer_index = None
# for i, layer in enumerate(efficient.layers):
#     if layer.name == 'block4c_add':
#         layer_index = i
#         break

In [57]:
# model = keras.applications.EfficientNetB0(include_top=True, weights='imagenet',input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[0], 3),pooling='avg')


In [58]:
# len(model.layers)

In [59]:
def create_vision_encoder(trainable_layer, trainable=False, input_shape=(139,139,3)):
    # Load ResNet50V2 model with lower computational complexity
    model = keras.applications.ResNet50V2(include_top=False, weights='imagenet', input_shape=input_shape)
    
    # Freeze all layers by default
    model.trainable = False
    
    # Optionally unfreeze layers up to trainable_layer
    if trainable:
        for layer in model.layers[-10:]:
            layer.trainable = trainable
    
    # Define input and apply preprocessing
    image = keras.Input(shape=input_shape, name="image")
    preprocessed = keras.applications.resnet_v2.preprocess_input(image)
    
    # Get the output from the model
    outputs = model(preprocessed)
    
    # Return the final model
    return keras.Model(inputs=[image], outputs=outputs, name="vision_encoder")


In [60]:
# def create_vision_encoder(trainable_layer,trainable=False,input_shape=(139,139,3)):
    
# #     resnet_v2 = keras.applications.ResNet50V2(include_top=False, weights="imagenet", pooling="max")
#     model = keras.applications.EfficientNetB0(include_top=False, weights='imagenet',input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[0], 3))
#     model.trainable= False
#     if trainable:
#         for layer in model.layers[:trainable_layer]:
#             layer.trainable = trainable
    
    
    
#     image = keras.Input(shape=input_shape, name="image")
    
#     preprocessed = keras.applications.efficientnet.preprocess_input(image)
    
    
#     outputs = model(preprocessed)
# #     outputs = model.get_layer(name='block4c_add').output
# #     outputs = model.layers[118].output
# #     outputs = Flatten()(outputs)
# #     outputs = layers.GlobalAveragePooling2D()(outputs)
#     return keras.Model(inputs=[image], outputs=outputs, name="vision_encoder")

In [61]:
# IMAGE_SIZE[0]

In [62]:
# vision_encoder = create_vision_encoder(trainable_layer=100,input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[0],3),trainable=True)
# len(vision_encoder.layers)

In [63]:
def vision_model():
    vision_encoder = create_vision_encoder(trainable_layer=235,input_shape=(IMAGE_SIZE,IMAGE_SIZE,3),trainable=True)
    
    x = layers.MaxPooling2D()(vision_encoder.output)
    x = layers.Flatten()(x)
    x = layers.Dense(128,activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(64,activation='relu')(x)
    x = layers.Dense(1,activation='sigmoid')(x)

    vision_model = keras.Model(inputs = vision_encoder.input, outputs=x)
    vision_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=[PartialAUC()])
    return vision_model

In [64]:
model = vision_model()

94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [65]:
tf.keras.backend.clear_session()

In [66]:
# from tensorflow.keras.mixed_precision import experimental as mixed_precision
# policy = mixed_precision.Policy('mixed_float16')
# mixed_precision.set_policy(policy)


In [67]:
# from tensorflow.keras import backend as K
# K.clear_session()

In [68]:
# # Check the length of the DataFrameIterator
# print("Number of batches:", len(train_2018))

# # Fetch a batch to inspect
# batch = next(train_2018)
# print("Batch shape (images):", batch[0].shape)
# print("Batch shape (labels):", batch[1].shape)

# # Try a simple fit to see if the data passes through correctly
# try:
#     history = model.fit(train_2018, epochs=5)
# except Exception as e:
#     print("Error:", e)


In [69]:
# history = model.fit(train_2018, epochs=5)

In [70]:
# history = model.fit(train_2019,validation_data=val_2019, epochs=5)

In [71]:
# history = model.fit(train_2020,validation_data=val_2020, epochs=5)

In [72]:
history = model.fit(train_data, epochs=10)

Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  1/258 ━━━━━━━━━━━━━━━━━━━━ 1:47:06 25s/step - loss: 1.2894 - partial_auc: 0.0552

I0000 00:00:1723225497.775141      71 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1723225497.824111      71 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


 86/258 ━━━━━━━━━━━━━━━━━━━━ 51s 300ms/step - loss: 0.5678 - partial_auc: 6.4154e-04

W0000 00:00:1723225523.205001      72 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


258/258 ━━━━━━━━━━━━━━━━━━━━ 72s 181ms/step - loss: 0.3618 - partial_auc: 0.0132
Epoch 2/10
258/258 ━━━━━━━━━━━━━━━━━━━━ 19s 71ms/step - loss: 0.1957 - partial_auc: 0.0376
Epoch 3/10
258/258 ━━━━━━━━━━━━━━━━━━━━ 19s 72ms/step - loss: 0.1849 - partial_auc: 0.0494
Epoch 4/10
258/258 ━━━━━━━━━━━━━━━━━━━━ 19s 73ms/step - loss: 0.1852 - partial_auc: 0.0649
Epoch 5/10
258/258 ━━━━━━━━━━━━━━━━━━━━ 20s 74ms/step - loss: 0.1800 - partial_auc: 0.0789
Epoch 6/10
258/258 ━━━━━━━━━━━━━━━━━━━━ 20s 75ms/step - loss: 0.1738 - partial_auc: 0.0802
Epoch 7/10
258/258 ━━━━━━━━━━━━━━━━━━━━ 20s 76ms/step - loss: 0.1674 - partial_auc: 0.0922
Epoch 8/10
258/258 ━━━━━━━━━━━━━━━━━━━━ 20s 75ms/step - loss: 0.1686 - partial_auc: 0.1001
Epoch 9/10
258/258 ━━━━━━━━━━━━━━━━━━━━ 21s 81ms/step - loss: 0.1749 - partial_auc: 0.0930
Epoch 10/10
258/258 ━━━━━━━━━━━━━━━━━━━━ 20s 75ms/step - loss: 0.1702 - partial_auc: 0.1015


In [73]:
import h5py


In [74]:


# # Load the HDF5 file
hdf5_file_path = '/kaggle/input/isic-2024-challenge/test-image.hdf5'
# hdf5_file = h5py.File(hdf5_file_path, 'r')

# # Explore the structure of the HDF5 file
# def print_hdf5_structure(hdf5_file):
#     for key in hdf5_file.keys():
#         print(f"Key: {key}")
#         print(f"Shape: {hdf5_file[key].shape}")
#         print(f"Datatype: {hdf5_file[key].dtype}")

# # Print the structure
# print_hdf5_structure(hdf5_file)

# # Close the file
# hdf5_file.close()


In [75]:
hdf5_file = h5py.File(hdf5_file_path, 'r')

# Function to preprocess the image
def preprocess_image(image_bytes):
    # Decode the image from the byte string
    image = tf.io.decode_jpeg(image_bytes)
    
    # Resize to the required input size of your model
    image = tf.image.resize(image, [224, 224])  # Adjust size as per your model
    
    # Normalize the image (if required)
    image = image / 255.0
    
    return image

# Prepare the images and their corresponding IDs
image_list = []
image_ids = []

for key in hdf5_file.keys():
    image_bytes = hdf5_file[key][()]
    image = preprocess_image(image_bytes)
    
    image_list.append(image)
    image_ids.append(key)

# Convert the list of images to a TensorFlow tensor
images_tensor = tf.stack(image_list)

# You now have the images ready for prediction
# Example: predictions = model.predict(images_tensor)

# Make sure to close the file
hdf5_file.close()

In [76]:
submission = pd.read_csv('/kaggle/input/isic-2024-challenge/sample_submission.csv')
submission

,isic_id,target
0,ISIC_0015657,0.3
1,ISIC_0015729,0.3
2,ISIC_0015740,0.3


In [77]:
# Assuming `model` is your trained TensorFlow model

# Make predictions
predictions = model.predict(images_tensor)

# Assuming binary classification, get the predicted class
predicted_classes = np.argmax(predictions, axis=1)

# Create a DataFrame for submission
# import pandas as pd

submission_df = pd.DataFrame({
    'isic_id': image_ids,
    'target': predicted_classes
})

# Save to a CSV file for submission
submission_df.to_csv('submission.csv', index=False)


1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step


W0000 00:00:1723225731.071587      72 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


In [78]:
submission_df

,isic_id,target
0,ISIC_0015657,0
1,ISIC_0015729,0
2,ISIC_0015740,0


In [79]:
# shuffled_dataset = dataset.shuffle(len(dataset))
# num_test_samples = int(len(dataset) * 0.2)  # Assuming 20% of the dataset for testing
# test_dataset = shuffled_dataset.take(num_test_samples)
# train_dataset = shuffled_dataset.take(int(len(dataset) * 0.8))

In [80]:
# len(test_dataset)

In [81]:
# for image, label in dataset.take(1):
#     print(label)

In [82]:
# model = vision_model()
# history = model.fit(previous_dataset, epochs=1)

In [83]:
# skf = StratifiedKFold(n_splits=5)
# fold_no = 1
# results = []
# images = df_concat[['file_path']]
# target = df_concat[['target']]

# for train_index, val_index in skf.split(images, target):
#     print(f'Fold {fold_no}...')
    
#     X_train, X_val = images.iloc[train_index], images.iloc[val_index]
#     y_train, y_val = target.iloc[train_index], target.iloc[val_index]
    
#     train_df = pd.concat([X_train, y_train],axis=1)
#     test_df = pd.concat([X_val, y_val],axis=1)
#     print(train_df.shape,test_df.shape)
    
#     vision_train_ds = create_dataset(train_df, IMAGE_SIZE, BATCH_SIZE,training=True,return_type='images')
#     vision_test_ds = create_dataset(test_df, IMAGE_SIZE, BATCH_SIZE,return_type='images')
#     # Instantiate the model for each fold
#     model = vision_model()
    
#     # Train the model
#     history = model.fit(vision_train_ds, epochs=2, validation_data=vision_test_ds,class_weight=class_weight)
    
#     # Evaluate the model on the validation set
#     y_true,y_pred = unbatch_test_dataset(vision_test_ds,model)
#     score = compute_partial_auc(y_true,y_pred, min_tpr=0.80)
# #     scores = model.evaluate(vision_test_ds, verbose=0)
#     print(f'Validation auc: {score}')
    
#     results.append(score)
#     fold_no += 1

# # Calculate the average performance
# # average_score = np.mean(results, axis=0)
# # print(f'\nAverage validation loss: {average_score[0]} - Average validation accuracy: {average_score[1]}')


### Metadata Transformations 

In [84]:
# df_concat.columns

In [85]:
# drop_columns = [
#     'isic_id', 'patient_id', 'image_type', 'tbp_tile_type', 'attribution', 
#     'copyright_license', 'lesion_id', 'iddx_full', 'iddx_1', 'iddx_2', 
#     'iddx_3', 'iddx_4', 'iddx_5', 'mel_mitotic_index', 'mel_thick_mm', 
#     'tbp_lv_dnn_lesion_confidence'
# ]
# df_concat = df_concat.drop(drop_columns,axis=1)

In [86]:
# df_concat.shape

In [87]:
# df_concat.head()

In [88]:
# df_concat.dtypes

In [89]:
# df_concat  = encode(df_concat,['sex','anatom_site_general','tbp_lv_location','tbp_lv_location_simple'])

In [90]:
# df_concat.head()

In [91]:
# df_concat.columns

In [92]:
# columns = ['age_approx', 'sex', 'anatom_site_general',
#        'clin_size_long_diam_mm', 'tbp_lv_A', 'tbp_lv_Aext', 'tbp_lv_B',
#        'tbp_lv_Bext', 'tbp_lv_C', 'tbp_lv_Cext', 'tbp_lv_H', 'tbp_lv_Hext',
#        'tbp_lv_L', 'tbp_lv_Lext', 'tbp_lv_areaMM2', 'tbp_lv_area_perim_ratio',
#        'tbp_lv_color_std_mean', 'tbp_lv_deltaA', 'tbp_lv_deltaB',
#        'tbp_lv_deltaL', 'tbp_lv_deltaLB', 'tbp_lv_deltaLBnorm',
#        'tbp_lv_eccentricity', 'tbp_lv_location', 'tbp_lv_location_simple',
#        'tbp_lv_minorAxisMM', 'tbp_lv_nevi_confidence', 'tbp_lv_norm_border',
#        'tbp_lv_norm_color', 'tbp_lv_perimeterMM',
#        'tbp_lv_radial_color_std_max', 'tbp_lv_stdL', 'tbp_lv_stdLExt',
#        'tbp_lv_symm_2axis', 'tbp_lv_symm_2axis_angle', 'tbp_lv_x', 'tbp_lv_y',
#        'tbp_lv_z']

In [93]:
# df_concat = scale_columns(df_concat,columns)

In [94]:
# df_concat.head()

In [95]:
# train_df, test_df = train_test_split(df_concat, test_size=0.11, 
#                                      stratify=df_concat["target"].values, random_state=42)
# # 5% for validation
# # train_df, val_df = train_test_split(train_df, test_size=0.11,stratify=train_df["target"].values, random_state=42)

# print(f"Total training examples: {len(train_df)}")
# # print(f"Total validation examples: {len(val_df)}")
# print(f"Total test examples: {len(test_df)}")

In [96]:
# labels = df['target']

In [97]:
# labels.shape

https://github.com/sayakpaul/Multimodal-Entailment-Baseline/blob/main/multimodal_entailment.ipynb

In [98]:
# class ProjectEmbeddings(keras.layers.Layer):
#     def __init__(self, num_projection_layers, projection_dims, dropout_rate, **kwargs):
#         super(ProjectEmbeddings, self).__init__(**kwargs)
#         self.num_projection_layers = num_projection_layers
#         self.projection_dims = projection_dims
#         self.dropout_rate = dropout_rate
        
#         # Initialize Dense and Dropout layers in __init__ or build
#         self.projection_layers = [layers.Dense(self.projection_dims) for _ in range(self.num_projection_layers)]
#         self.dropout_layers = [layers.Dropout(self.dropout_rate) for _ in range(self.num_projection_layers - 1)]

#     def call(self, embeddings):
#         x = embeddings
#         for i in range(self.num_projection_layers):
#             x = self.projection_layers[i](x)
#             if i < self.num_projection_layers - 1:
#                 x = tf.nn.gelu(x)
#                 x = self.dropout_layers[i](x)
#         return x

In [99]:
# df_concat.shape

(7860, 56), negative shape: (11790, 56)

In [100]:
# vision_test_ds

In [101]:
# # Create the dataset
# vision_train_ds = create_dataset(train_df, IMAGE_SIZE, batch_size,training=True,return_type='images')
# vision_test_ds = create_dataset(test_df, IMAGE_SIZE, batch_size,return_type='images')
# # val_ds = create_dataset(val_df, image_size, batch_size)

In [102]:
# vision_history = vision_model.fit(
#     train_ds,
#     validation_data=test_ds,
#     epochs=20,
#     class_weight=class_weight,
#     callbacks=[partial_auc_callback]
# )

In [103]:
# for (images, metadata), labels in train_ds.take(1):
#     print(images.shape,metadata.shape,labels.shape)

In [104]:
# !pip install -q focal-loss

In [105]:
# from focal_loss import BinaryFocalLoss

In [106]:
# def create_table_encoder(
#     num_projection_layers, projection_dims, dropout_rate, trainable=False
# ):
#     table_input = tf.keras.Input(shape=(38,))
#     dense_1024 = tf.keras.layers.Dense(1024, activation='relu')(table_input)
#     dense_512 = tf.keras.layers.Dense(512, activation='relu')(dense_1024)

#     # Project the embeddings produced by the model.
#     outputs = ProjectEmbeddings(num_projection_layers, projection_dims, dropout_rate)(dense_512)

#     # Create the text encoder model.
#     return keras.Model(table_input, outputs, name="table_encoder")

In [107]:
# table_encoder = create_table_encoder(num_projection_layers=1, projection_dims=256, dropout_rate=0.1)
# # keras.utils.plot_model(table_encoder, show_shapes=True)

In [108]:
# def create_multimodal_model(num_projection_layers=1, projection_dims=256, dropout_rate=0.1, 
#                      vision_trainable=False, text_trainable=False):
#     # Receive the images as inputs.
#     image_1 = keras.Input(shape=(139, 139, 3), name="image_1")
#     table_input = tf.keras.Input(shape=(38,))

    

#     # Create the encoders.
#     vision_encoder = create_vision_encoder(num_projection_layers, projection_dims, dropout_rate, vision_trainable)
#     table_encoder = create_table_encoder(num_projection_layers, projection_dims, dropout_rate, text_trainable)

#     # Fetch the embedding projections.
#     vision_projections = vision_encoder(image_1)
#     table_projections = table_encoder(table_input)

#     # Concatenate the projections and pass through the classification layer.
#     concatenated = keras.layers.Concatenate()([vision_projections, table_projections])
#     outputs = keras.layers.Dense(1, activation="sigmoid")(concatenated)
#     return keras.Model([image_1, table_input], outputs)

In [109]:
# vision_model = create_vision_encoder(2, 256, 0.5,True)
# keras.utils.plot_model(vision_model, show_shapes=True)

In [110]:
# vision_model.output

In [111]:

# multimodal_model = create_multimodal_model(vision_trainable=True, text_trainable=True)
# keras.utils.plot_model(multimodal_model, show_shapes=True)

In [112]:
# multimodal_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=METRICS)


In [113]:
# history = multimodal_model.fit(
#     train_ds,
#     validation_data=test_ds,
#     epochs=10,
#     class_weight=class_weight
# )

In [114]:
# from focal_loss import BinaryFocalLoss

In [115]:
# multimodal_model_focal = create_multimodal_model()
# multimodal_model_focal.compile(
#     optimizer="adam", loss=BinaryFocalLoss(gamma=2), metrics=METRICS
# )
# history = multimodal_model_focal.fit(
#     train_ds,
#     validation_data=test_ds,
#     epochs=10
# )